In [16]:
#Project 1
#Chase Scott
#Collaborators: Issadore Sossa
#Resources:
#     Class notes
#     Nurse Assignment
#     Canvas Links

In [17]:
import pandas as pd
from docplex.mp.model import Model
import math
import random
from datetime import datetime, timedelta

In [22]:
## Load data
data_file = 'Data Set – Class Schedule.xlsx'
file_handler = pd.ExcelFile(io=data_file)
file_sheets = sorted(file_handler.sheet_names)

print(f'Sheets : {file_sheets}')

## Load each sheet as a DataFrame
classrooms_df = pd.read_excel(io=file_handler, sheet_name=file_sheets[0])
display(file_sheets[0], classrooms_df.head(5))

colleges_and_depts = pd.read_excel(io=file_handler, sheet_name=file_sheets[1])
display(file_sheets[1], colleges_and_depts.head(5))

course_catalog_df = pd.read_excel(io=file_handler, sheet_name=file_sheets[2])
display(file_sheets[2], course_catalog_df.head(5))

courses_offered = pd.read_excel(io=file_handler, sheet_name=file_sheets[3])
display(file_sheets[3], courses_offered.head(5))

professors_df = pd.read_excel(io=file_handler, sheet_name=file_sheets[4])
display(file_sheets[4], professors_df.head(5))

qualifications_df = pd.read_excel(io=file_handler, sheet_name=file_sheets[5])
display(file_sheets[5], qualifications_df.head(5))

# For reproducibility
random.seed(1345)

Sheets : ['Classrooms', 'Colleges and Departments', 'Course Catalog', 'Courses Offered Fall 2021', 'Professors', 'Qualification']


'Classrooms'

,Building,Building Code,Classroom,Capacity
0,Durland,DU,1029,47
1,Durland,DU,1041,24
2,Durland,DU,1052,60
3,Durland,DU,1061,47
4,Durland,DU,1063,47


'Colleges and Departments'

,College,College Code,Department,Department Code
0,College of Agriculture,CAGRI,Entomology,ENTOM


...

,Department Code,Course Number,Course Name,Sections,Day,Max Enrollement
0,ENTOM,300,Economic Entomology,1,M/W/F,50
1,ENTOM,301,Insects and People,3,TU/TH,50
2,ENTOM,305,Animal Health Entomology,3,TU/TH,50
3,ENTOM,306,Animal Health Entomology Laboratory,1,M/W/F,50
4,ENTOM,350,"Crops, Insects, and Agroecosystems",1,M/W,50


'Professors'

,Department Code,Faculty Name,Workload Credit Hours
0,ENTOM,Frank H. Arthur,6
1,ENTOM,James F. Campbell,9
2,ENTOM,Ming-Shun Chen,9
3,ENTOM,Raymond Cloyd,9
4,ENTOM,Lee Cohnstaedt,9


'Qualification'

,Department Code,Faculty Name,ENTOM 300,ENTOM 301,ENTOM 305,ENTOM 306,ENTOM 350,ENTOM 589,ENTOM 602,ENTOM 621,...,ENTOM 830,ENTOM 835,ENTOM 837,ENTOM 840,ENTOM 849,ENTOM 857,ENTOM 860,ENTOM 875,ENTOM 880,ENTOM 885
0,ENTOM,Frank H. Arthur,NaN,1.0,1.0,NaN,1.0,NaN,NaN,NaN,...,1.0,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN
1,ENTOM,James F. Campbell,NaN,1.0,1.0,NaN,1.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,1.0
2,ENTOM,Ming-Shun Chen,NaN,1.0,1.0,1.0,NaN,1.0,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0
3,ENTOM,Raymond Cloyd,NaN,1.0,1.0,1.0,1.0,1.0,NaN,NaN,...,NaN,1.0,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN
4,ENTOM,Lee Cohnstaedt,1.0,1.0,1.0,1.0,1.0,NaN,1.0,NaN,...,1.0,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN


In [23]:
## Duplicate courses with multiple sessions
def standardize_courses_offered(courses_offered:pd.DataFrame, course_catalog_df:pd.DataFrame) -> pd.DataFrame:
    result = pd.DataFrame()
    result = result.append(courses_offered[courses_offered['Sections'] == 1])
    for course in courses_offered[courses_offered['Sections'] > 1].itertuples():
        course = pd.DataFrame(course).T.drop(columns=0)
        course.columns = courses_offered.columns
        for i in range(1, course['Sections'][0] + 1):
            course['Sections'] = i
            result = result.append([course], ignore_index=True)
    result = result.rename(columns={'Sections' : 'Section Number'})
    result = result.sort_values(by=['Course Number', 'Section Number']).reset_index().drop(columns='index')
    result = pd.merge(left=result, right=course_catalog_df, on=['Department Code', 'Course Number', 'Course Name'])
    return result

standardize_courses_offered(courses_offered, course_catalog_df)

,Department Code,Course Number,Course Name,Section Number,Day,Max Enrollement,Credit Hours
0,ENTOM,300,Economic Entomology,1,M/W/F,50,3
1,ENTOM,301,Insects and People,1,TU/TH,50,3
2,ENTOM,301,Insects and People,2,TU/TH,50,3
3,ENTOM,301,Insects and People,3,TU/TH,50,3
4,ENTOM,305,Animal Health Entomology,1,TU/TH,50,3
5,ENTOM,305,Animal Health Entomology,2,TU/TH,50,3
6,ENTOM,305,Animal Health Entomology,3,TU/TH,50,3
7,ENTOM,306,Animal Health Entomology Laboratory,1,M/W/F,50,3
8,ENTOM,350,"Crops, Insects, and Agroecosystems",1,M/W,50,3
9,ENTOM,589,Turfgrass Insects and Their Management,1,M/W/F,40,3


In [27]:
## Clean
def merge_and_get_preference(professors_df:pd.DataFrame, qualifications:pd.DataFrame, course:str) -> pd.DataFrame:
    ''''''
    def manipulate_row(row:pd.DataFrame, course:str):
        cols = [x for x in row.columns if course in x]
        class_taught = 0
        for col in cols:
            if not math.isnan(row[col][0]):
                class_taught += row[col][0]
        class_taught = int(class_taught)
        if row['Workload Credit Hours'][0] > 0:
            preferences = random.sample(list(range(1, class_taught + 1)), class_taught)
        else:
            preferences = [0] * class_taught
            
        #display(preferences)

        index = 0
        for col in cols:
            if not math.isnan(row[col][0]):
                row[col] = preferences[index]
                index += 1
            else:
                row[col] = 0
        return row

    result_df = pd.merge(left=professors_df, right=qualifications_df, on=['Faculty Name', 'Department Code'])
    col_names = list(result_df.columns)
    preferences_df = pd.DataFrame(columns=col_names)
    col_names = result_df.columns
    for row in result_df.itertuples():
        row_df = pd.Series(row).to_frame().T.drop(columns=0)
        row_df.columns = col_names
        row_df = manipulate_row(row_df, course)
        preferences_df = preferences_df.append(row_df, ignore_index=True)
    return preferences_df

preferences_df = merge_and_get_preference(professors_df, qualifications_df, 'ENTOM')
display(preferences_df)


,Department Code,Faculty Name,Workload Credit Hours,ENTOM 300,ENTOM 301,ENTOM 305,ENTOM 306,ENTOM 350,ENTOM 589,ENTOM 602,...,ENTOM 830,ENTOM 835,ENTOM 837,ENTOM 840,ENTOM 849,ENTOM 857,ENTOM 860,ENTOM 875,ENTOM 880,ENTOM 885
0,ENTOM,Frank H. Arthur,6,0,2,8,0,5,0,0,...,3,0,0,4,1,0,0,0,0,0
1,ENTOM,James F. Campbell,9,0,6,8,0,4,1,0,...,0,0,0,0,3,0,0,0,7,5
2,ENTOM,Ming-Shun Chen,9,0,6,2,3,0,8,0,...,0,0,0,0,0,0,0,1,7,5
3,ENTOM,Raymond Cloyd,9,0,4,11,3,2,8,0,...,0,9,0,0,10,7,0,0,0,0
4,ENTOM,Lee Cohnstaedt,9,6,3,4,5,10,0,2,...,11,0,0,0,7,0,9,0,0,0
5,ENTOM,Srinivas Kambhampati,6,0,0,6,3,0,0,0,...,0,0,0,9,0,0,7,0,8,0
6,ENTOM,Tania Kim,3,7,0,3,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,ENTOM,Berlin Luxelly Londono Renteria,6,4,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
8,ENTOM,Jeremy L. Marshall,9,7,3,2,6,0,0,8,...,0,0,0,0,0,0,0,0,0,0
9,ENTOM,Brian P. McCornack,9,0,0,0,1,0,0,0,...,3,2,0,5,7,0,0,0,0,0


In [28]:
def get_period_df() -> pd.DataFrame:
    '''Create a dataframe with days'''
    days = ['Monday', 'Tuesday', 'Wednesday', 'Thrusday', 'Friday']
    days_of_weeks = dict(zip(days, range(1, 6)))
    days_of_weeks

    result = pd.DataFrame(columns=['Period', 'Day'])
    start = 1
    day_digits = []
    day_names = []
    for day, day_digit in days_of_weeks.items():
        day_names.extend([day] * 20)
        day_digits.extend([day_digit] * 20)
        
    start_hour = 8
    end_hour = 18
    start_time = [x * 0.5 for x in range(2*start_hour, 2*end_hour)] * len(days)
    end_time = [x * 0.5 for x in range(2*start_hour + 1, 2*end_hour + 1)] * len(days)
    
    
    return pd.DataFrame.from_dict({'Period' : list(range(1, 21)) * 5, 'Day Name' : day_names, 'Day Number' : day_digits, 
                                   'Start Time' : start_time, 'End Time' : end_time})

get_period_df().tail(60)

,Period,Day Name,Day Number,Start Time,End Time
40,1,Wednesday,3,8.0,8.5
41,2,Wednesday,3,8.5,9.0
42,3,Wednesday,3,9.0,9.5
43,4,Wednesday,3,9.5,10.0
44,5,Wednesday,3,10.0,10.5
45,6,Wednesday,3,10.5,11.0
46,7,Wednesday,3,11.0,11.5
47,8,Wednesday,3,11.5,12.0
48,9,Wednesday,3,12.0,12.5
49,10,Wednesday,3,12.5,13.0


In [9]:
## Model Environment
model = Model(name='Course Scheduling')
model.print_information()

## Decision Variables

## Constraints

    #Need
        #Capacity
        #Time start no earlier than 0800
        #Time end no later than 1800
        #Every course must be assigned
        #no section  can  be  assigned  to  more  than  one  professor

    #Rule 1
        #Prof doesn't have back to back course
            #Do by sum of consecutive periods with a break of some arbitrary number
    #Rule 2
        #No scheduled classes after noon on Friday



## Objective Function

total_preferance = sum(preferance_df )

mdl.maximize(total_preferance)

Model: Course Scheduling
 - number of variables: 0
   - binary=0, integer=0, continuous=0
 - number of constraints: 0
   - linear=0
 - parameters: defaults
 - objective: none
 - problem type is: LP
